# Creación de las Funciones

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error

In [2]:
df_search = pd.read_json('data_search.json')
df_search.head()

,genres,title,release_date,specs,early_access,sentiment,metascore,date1,date2
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,[Single-player],False,None,None,NaN,NaN
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",False,Mostly Positive,None,NaN,NaN
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",False,Mostly Positive,None,NaN,NaN
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,[Single-player],False,None,None,NaN,NaN
4,None,None,None,"[Single-player, Full controller support, HTC V...",False,None,None,NaN,NaN


In [3]:
genre = "['Action', 'Casual', 'Indie', 'Simulation', 'Strategy']"

In [4]:
df_search.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 0 to 32134
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28852 non-null  object 
 1   title         30085 non-null  object 
 2   release_date  29894 non-null  object 
 3   specs         31465 non-null  object 
 4   early_access  32135 non-null  bool   
 5   sentiment     24953 non-null  object 
 6   metascore     2677 non-null   object 
 7   date1         103 non-null    float64
 8   date2         14 non-null     float64
dtypes: bool(1), float64(2), object(6)
memory usage: 2.2+ MB


In [5]:
df_search['release_date'] = pd.to_datetime(df_search["release_date"], errors='coerce')
df_search['metascore'] = pd.to_numeric(df_search['metascore'], errors='coerce')
df_search.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 0 to 32134
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28852 non-null  object        
 1   title         30085 non-null  object        
 2   release_date  29894 non-null  datetime64[ns]
 3   specs         31465 non-null  object        
 4   early_access  32135 non-null  bool          
 5   sentiment     24953 non-null  object        
 6   metascore     2607 non-null   float64       
 7   date1         103 non-null    float64       
 8   date2         14 non-null     float64       
dtypes: bool(1), datetime64[ns](1), float64(3), object(4)
memory usage: 2.2+ MB


**`Desarrollo API`**:   Propones disponibilizar los datos de la empresa usando el framework ***FastAPI***. Las consultas que propones son las siguientes:

Deben crear 6 funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

+ def **genero( *`Año`: str* )**:
    Se ingresa un año y devuelve una lista con los 5 géneros más vendidos en el orden correspondiente.

+ def **juegos( *`Año`: str* )**:
    Se ingresa un año y devuelve una lista con los juegos lanzados en el año.

+ def **specs( *`Año`: str* )**:
    Se ingresa un año y devuelve una lista con los 5 specs que más se repiten en el mismo en el orden correspondiente. 

+ def **earlyacces( *`Año`: str* )**:
    Cantidad de juegos lanzados en un año con early access.

+ def **sentiment( *`Año`: str* )**:
    Según el año de lanzamiento, se devuelve una lista con la cantidad de registros que se encuentren categorizados con un análisis de sentimiento. 

    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ejemplo de retorno: *{Mixed = 182, Very Positive = 120, Positive = 278}*

+ def **metascore( *`Año`: str* )**:
    Top 5 juegos según año con mayor metascore.




In [6]:
def find_year(anio):
    """Función de soporte para las demas funciones, recibe un año (int)
       y devuelve un dataframe solo con los valores de ese año"""
    df_anio = df_search[df_search['release_date'].dt.year == anio]
    return df_anio


def genero(anio):
    """Recibe un año (int) y devuelve una lista con los 5 géneros
       más vendidos en el orden correspondiente."""
    df_genero = find_year(anio)
    df_genero = df_genero.explode("genres")
    lista_generos = df_genero['genres'].value_counts().head().index.to_list()
    return lista_generos


def juegos(anio):
    """Recibe un año (int) y devuelve una lista con los juegos lanzados en el año."""
    df_juegos = find_year(anio)
    lista_juegos = df_juegos.title.to_list()
    return lista_juegos


def specs(anio):
    """Recibe un año (int) y devuelve una lista con los 5 specs que 
       más se repiten en el mismo en el orden correspondiente."""
    df_specs = find_year(anio)
    df_specs = df_specs.explode("specs")
    lista_specs = df_specs['specs'].value_counts().head().index.to_list()
    return lista_specs


def earlyacces(anio):
    """Recibe un año (int) y devuelve la cantidad de juegos lanzados en ese año con early access."""
    df_early = find_year(2019)
    early = df_early['early_access'].sum()
    return early


def sentiment(anio):
    """Recibe un año (int) y se devuelve una lista con la cantidad de registros que
       se encuentren categorizados con un análisis de sentimiento ese año."""
    df_sentiment = find_year(anio)
    sent_on = (df_sentiment["sentiment"] == 'Overwhelmingly Negative').sum()
    sent_vn = (df_sentiment["sentiment"] == 'Very Negative').sum()
    sent_n  = (df_sentiment["sentiment"] == 'Negative').sum()
    sent_mn = (df_sentiment["sentiment"] == 'Mostly Negative').sum()
    sent_m  = (df_sentiment["sentiment"] == 'Mixed').sum()
    sent_mp = (df_sentiment["sentiment"] == 'Mostly Positive').sum()
    sent_p  = (df_sentiment["sentiment"] == 'Positive').sum()
    sent_vp = (df_sentiment["sentiment"] == 'Very Positive').sum()
    sent_op = (df_sentiment["sentiment"] == 'Overwhelmingly Positive').sum()

    sent_on_str = f"Overwhelmingly Negative: {sent_on}"
    sent_vn_str = f"Very Negative: {sent_vn}"
    sent_n_str  = f"Negative: {sent_n}"
    sent_mn_str = f"Mostly Negative: {sent_mn}"
    sent_m_str  = f"Mixed: {sent_m}"
    sent_mp_str = f"Mostly Positive: {sent_mp}"
    sent_p_str  = f"Positive: {sent_p}"
    sent_vp_str = f"Very Positive: {sent_vp}"
    sent_op_str = f"Overwhelmingly Positive: {sent_op}"

    lista = [[sent_on, sent_on_str], [sent_vn, sent_vn_str], [sent_n, sent_n_str], [sent_mn, sent_mn_str], [sent_m, sent_m_str],
             [sent_mp, sent_mp_str], [sent_p, sent_p_str], [sent_vp, sent_vp_str], [sent_op, sent_op_str]]

    lista_final = []

    for sent in lista:
        if sent[0] > 0:
            lista_final.append(sent[1])

    return lista_final


def metascore(anio):
    """Recibe un año (int) y retorna el top 5 juegos con mayor metascore."""
    df_meta = find_year(anio)
    df_meta = df_meta[['title', 'metascore']].sort_values('metascore', axis=0, ascending=False).head()

    lista_name_score = []

    for i in range(df_meta.shape[0]):
        name = df_meta.iloc[i:i+1, 0:1].values[0][0]
        score = df_meta.iloc[i:i+1, 1:2].values[0][0]
        name_score = f"{name}: {score}"
        lista_name_score.append(name_score)
    return lista_name_score


Función para la predicción:

In [7]:
def prediccion(early_access, sentiment, publisher, developer, year, month, genres, specs, tags, precio_real='0'):
    """Esta función recibe todas las variables necesarias, y devuelve la predicción del precio.
       Si se ingresa el precio real devuelve el RMSE según la predicción, si no devuelve el mejor
       RMSE general obtenido por el modelo usando Cross Validation"""
    
    # Cargamos el modelo
    with open('lgbm_regressor_model.pkl', 'rb') as modelo:
        modelo_lgbm = pickle.load(modelo)
    
    # Cargamos tabla vacía para predicción
    with open('x_prediccion.pkl', 'rb') as x_prediccion:
        x_pred = pickle.load(x_prediccion)

    # Diccionario con publishers, ya que están separados
    # en categorías de 0 a 5 segun pupularidad
    with open('dict_publishers.pkl', 'rb') as dict_pub:
        dict_publishers = pickle.load(dict_pub)

    # Diccionario con developers, ya que están separados
    # en categorías de 0 a 5 segun pupularidad
    with open('dict_developers.pkl', 'rb') as dict_dev:
        dict_developers = pickle.load(dict_dev)


    # Early_access es True o False
    x_pred['early_access'] = early_access


    # Sentiment está compuesto por números que van del 0 al 10
    # Recibimos un string y según eso colocamos el número correspondiente
    list_sent = ["Sin reviews",
                 "X user reviews",
                 "Overwhelmingly Negative",
                 "Very Negative",
                 "Negative",
                 "Mostly Negative",
                 "Mixed",
                 "Mostly Positive",
                 "Positive",
                 "Very Positive",
                 "Overwhelmingly Positive",
                 ]

    for i, sent in enumerate(list_sent):
        if sent == sentiment:
            x_pred['sentiment'] = i


    # Comparamos los nombres de los publishers con respecto
    #  a las listas y les asignamos un valor numérico.
    if publisher in dict_publishers['lista_pub1']:
        x_pred['publisher_cat'] = 1
    elif publisher in dict_publishers['lista_pub2']:
        x_pred['publisher_cat'] = 2
    elif publisher in dict_publishers['lista_pub3']:
        x_pred['publisher_cat'] = 3
    elif publisher in dict_publishers['lista_pub4']:
        x_pred['publisher_cat'] = 4
    elif publisher in dict_publishers['lista_pub5']:
        x_pred['publisher_cat'] = 5
    else:
        x_pred['publisher_cat'] = 0


    # Comparamos los nombres de los developers con respecto
    #  a las listas y les asignamos un valor numérico.
    if developer in dict_developers['lista_dev1']:
        x_pred['developer_cat'] = 1
    elif developer in dict_developers['lista_dev2']:
        x_pred['developer_cat'] = 2
    elif developer in dict_developers['lista_dev3']:
        x_pred['developer_cat'] = 3
    elif developer in dict_developers['lista_dev4']:
        x_pred['developer_cat'] = 4
    elif developer in dict_developers['lista_dev5']:
        x_pred['developer_cat'] = 5
    else:
        x_pred['developer_cat'] = 0

    
    
    x_pred['year'] = year

    # Las columnas month van de 2 a 12, si el mes es cero quedan 
    # todas en 0, de otra forma se coloca un 1 en el mes correspondiente.
    if month != 1:
        columna_month = f"month_{month}"
        x_pred[columna_month] = True

    # Hacemos lista de specs, tags y genres:
    lista_features = x_pred.columns.tolist()
    lista_features = lista_features[16:]

    # Completamos con 1 las columnas correspondientes
    for genre in genres:
        if genre.lower() in lista_features:
            x_pred[genre.lower()] = 1
    for spec in specs:
        if spec.lower() in lista_features:
            x_pred[spec.lower()] = 1
    for tag in tags:
        if tag.lower() in lista_features:
            x_pred[tag.lower()] = 1


    # Hacemos la predicción
    prediccion = modelo_lgbm.predict(x_pred)

    if precio_real == '0':
        rmse = f"RMSE del modelo: 8.144"
    else:
        rmse = f"RMSE: {np.sqrt(mean_squared_error(precio_real, prediccion))}"
    
    pred_str = f"Precio predicho {round(prediccion[0], 2)}"

    return {'prediccion': pred_str,
            'RMSE' : rmse}

In [8]:
list_sent = ["Sin reviews",
                 "N user reviews",
                 "Overwhelmingly Negative",
                 "Very Negative",
                 "Negative",
                 "Mostly Negative",
                 "Mixed",
                 "Mostly Positive",
                 "Positive",
                 "Very Positive",
                 "Overwhelmingly Positive",
                 ]

for sent, i in enumerate(list_sent):
    print (sent, i)

0 Sin reviews
1 N user reviews
2 Overwhelmingly Negative
3 Very Negative
4 Negative
5 Mostly Negative
6 Mixed
7 Mostly Positive
8 Positive
9 Very Positive
10 Overwhelmingly Positive
